-

<font size=10>**tensorflow2保存和加载模型**</font>

模型进度可以在训练期间和训练后保存。这意味着模型可以在它停止的地方继续，并避免长时间的训练。保存还意味着您可以共享您的模型，其他人可以重新创建您的工作。当发布研究模型和技术时，大多数机器学习实践者共享:

- 用于创建模型的代码
- 以及模型的训练权重或参数


# 1. 设置

- 1.1. 安装和导入
    - pip install h5py pyyaml
- 1.2. 获取样本数据集
- 1.3. 定义模型

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
 
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

In [2]:
# 我们将使用MNIST数据集来训练我们的模型以演示保存权重，要加速这些演示运行，请只使用前1000个样本数据：

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
#-------------------- 1.3. 定义模型

# 让我们构建一个简单的模型，我们将用它来演示保存和加载权重。

# 返回一个简短的序列模型 
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# 创建基本模型实例
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# 2. 在训练期间保存检查点

主要用例是在训练期间和训练结束时自动保存检查点，通过这种方式，您可以使用训练有素的模型，而无需重新训练，或者在您离开的地方继续训练，以防止训练过程中断。

tf.keras.callbacks.ModelCheckpoint是执行此任务的回调，回调需要几个参数来配置检查点。

- 2.1. 检查点回调使用情况
- 2.2. 检查点选项

## 2.1. 检查点回调使用情况
训练模型并将其传递给 ModelCheckpoint回调

In [4]:
base_path = "/Users/zhouwencheng/Desktop/Grass/data/model/201_601_AI/101_tf2.0_model_save"
checkpoint_path = f"{base_path}/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个检查点回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# 这可能会生成与保存优化程序状态相关的警告。
# 这些警告（以及整个笔记本中的类似警告）是为了阻止过时使用的，可以忽略。

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 800/1000 [=======================>......] - ETA: 0s - loss: 1.2493 - accuracy: 0.6513
Epoch 00001: saving model to /Users/zhouwencheng/Desktop/Grass/data/model/201_601_AI/101_tf2.0_model_save/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.1264 - accuracy: 0.6810 - val_loss: 0.6946 - val_accuracy: 0.7840
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4099 - accuracy: 0.8901
Epoch 00002: saving model to /Users/zhouwencheng/Desktop/Grass/data/model/201_601_AI/101_tf2.0_model_save/cp.ckpt
1000/1000 [==============================] - 0s 330us/sample - loss: 0.4094 - accuracy: 0.8890 - val_loss: 0.5281 - val_accuracy: 0.8320
Epoch 3/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.2875 - accuracy: 0.9214
Epoch 00003: saving model to /Users/zhouwencheng/Desktop/Grass/data/model/201_601_AI/101_tf2.0_model_save/cp.ckpt
1000/1000 [==============================] -

In [5]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Untrained model, accuracy:  6.20%


In [6]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Restored model, accuracy: 86.90%


## 2.2. 检查点选项

回调提供了几个选项，可以为生成的检查点提供唯一的名称，并调整检查点频率。

训练一个新模型，每5个周期保存一次唯一命名的检查点：

In [8]:
# 在文件名中包含周期数. (使用 `str.format`)
checkpoint_path = base_path+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 每5个周期保存一次权重
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 10, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to /Users/zhouwencheng/Desktop/Grass/data/model/201_601_AI/101_tf2.0_model_save/cp-0005.ckpt

Epoch 00010: saving model to /Users/zhouwencheng/Desktop/Grass/data/model/201_601_AI/101_tf2.0_model_save/cp-0010.ckpt


# 3. 这些文件是什么？
上述代码将权重存储到检查点)格式的文件集合中，这些文件仅包含二进制格式的训练权重.
检查点包含：

- 一个或多个包含模型权重的分片；
- 索引文件，指示哪些权重存储在哪个分片。

# 4. 手动保存权重
上面你看到了如何将权重加载到模型中。手动保存权重同样简单，使用Model.save_weights方法。

In [9]:
# 保存权重
model.save_weights(base_path+'/my_checkpoint')

# 加载权重
model = create_model()
model.load_weights(base_path+'/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Restored model, accuracy: 86.60%


# 5. 保存整个模型
模型和优化器可以保存到包含其状态（权重和变量）和模型配置的文件中，这允许您导出模型，以便可以在不访问原始python代码的情况下使用它。由于恢复了优化器状态，您甚至可以从中断的位置恢复训练。

保存完整的模型非常有用，您可以在TensorFlow.js(HDF5, Saved Model) 中加载它们，然后在Web浏览器中训练和运行它们，或者使用TensorFlow Lite(HDF5, Saved Model)将它们转换为在移动设备上运行。

- 5.1. 作为HDF5文件
- 5.2. 作为 saved_model


##  5.1. 作为HDF5文件
Keras使用HDF5标准提供基本保存格式，出于我们的目的，可以将保存的模型视为单个二进制blob。

In [10]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 保存整个模型到HDF5文件 
model.save(base_path+'/my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 812us/sample - loss: 1.1890 - accuracy: 0.6750
Epoch 2/5
1000/1000 [==============================] - 0s 204us/sample - loss: 0.4443 - accuracy: 0.8670
Epoch 3/5
1000/1000 [==============================] - 0s 174us/sample - loss: 0.2822 - accuracy: 0.9260
Epoch 4/5
1000/1000 [==============================] - 0s 173us/sample - loss: 0.2074 - accuracy: 0.9420
Epoch 5/5
1000/1000 [==============================] - 0s 191us/sample - loss: 0.1615 - accuracy: 0.9630


In [11]:
# 重新创建完全相同的模型，包括权重和优化器
new_model = keras.models.load_model(base_path+'/my_model.h5')
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [12]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Restored model, accuracy: 86.80%


此方法可保存模型的所有东西：

- 权重值
- 模型的配置（架构）
- 优化器配置

## 5.2. 作为 saved_model
注意：这种保存tf.keras模型的方法是实验性的，在将来的版本中可能会有所改变。 

In [13]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 871us/sample - loss: 1.1818 - accuracy: 0.6550
Epoch 2/5
1000/1000 [==============================] - 0s 209us/sample - loss: 0.4310 - accuracy: 0.8790
Epoch 3/5
1000/1000 [==============================] - 0s 175us/sample - loss: 0.2907 - accuracy: 0.9170
Epoch 4/5
1000/1000 [==============================] - 0s 177us/sample - loss: 0.2146 - accuracy: 0.9550
Epoch 5/5
1000/1000 [==============================] - 0s 175us/sample - loss: 0.1542 - accuracy: 0.9700


In [14]:
# 创建saved_model，并将其放在带时间戳的目录中：

import time
saved_model_path = base_path+"/saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

'/Users/zhouwencheng/Desktop/Grass/data/model/201_601_AI/101_tf2.0_model_save/saved_models/1570686912'

In [15]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
